In [1]:
%matplotlib inline
from PIL import Image
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
import os
import itertools
from tqdm import *
from tqdm import tqdm_notebook as tqdm
import random
import numpy as np
import math

from IPython.display import clear_output

# Step 1: Load Data Set

In [2]:
fileNames = []
for file in os.listdir("data/train_1"):
    if file.endswith(".jpg"):
        fileNames.append(file)
# fileNamesDF = pd.DataFrame(fileNames)

In [3]:
#only take certain artist so that the probabilities aren't basically all false when doing the tupples
minNumPaintingsPerArtist = 150

# #for each artist, only take n of their paintings
numPaintingsPerAuthor = 100


trainInfo = pd.read_csv('data/train_info.csv')
mostPopularArtists = trainInfo['artist'].value_counts()[trainInfo['artist'].value_counts() > minNumPaintingsPerArtist]

trainInfo = trainInfo[trainInfo['filename'].isin(fileNames)]
fileNamesDFAll = trainInfo[trainInfo['artist'].isin(mostPopularArtists.index)]


fileNamesDF = pd.DataFrame()
for artist in tqdm(mostPopularArtists.index):
    fileNamesDF = fileNamesDF.append(trainInfo[trainInfo['artist'] == artist][:numPaintingsPerAuthor])
    
fileNamesDF = fileNamesDF['filename']

In [4]:
# Load the all_data_info for the dataset we are using
allInfo = pd.read_csv('data/all_data_info.csv')
allInfo = allInfo[allInfo['new_filename'].isin(fileNamesDF)]

In [5]:
featuresDF = pd.DataFrame(allInfo['new_filename'])
featuresDF['pixelsx'] = allInfo['pixelsx']
featuresDF['pixelsy'] = allInfo['pixelsy']

In [6]:
def normalizeSeries(series, maxNum):
    series = series/maxNum
    return series

In [7]:
def featurizeImage(filename):
    # Extract the features from the actual image
    jpgfile = Image.open("data/train_1/" + filename)
#     print(jpgfile)
    
    bands = jpgfile.getbands()
    if bands[0] == 'L':
        # grayscale image
        imageDF = pd.DataFrame(list(jpgfile.getdata()),columns=['grayscale'])
#         imageDF['grayscale'] = normalizeSeries(imageDF['grayscale',])
    elif len(bands) == 4:
        #get red, green and blue chanels 
        imageDF = pd.DataFrame(list(jpgfile.getdata()),columns=['red','green','blue','alpha'])
        imageDF['red'] = normalizeSeries(imageDF['red'],255)
        imageDF['green'] = normalizeSeries(imageDF['green'],255)
        imageDF['blue'] = normalizeSeries(imageDF['blue'],255)  
        imageDF['alpha'] = normalizeSeries(imageDF['alpha'],255)  
    elif len(bands) == 3:
        #get red, green and blue chanels 
        imageDF = pd.DataFrame(list(jpgfile.getdata()),columns=['red','green','blue'])
        imageDF['red'] = normalizeSeries(imageDF['red'],255)
        imageDF['green'] = normalizeSeries(imageDF['green'],255)
        imageDF['blue'] = normalizeSeries(imageDF['blue'],255)  
    else:
        imageDF = pd.DataFrame(list(jpgfile.getdata()))
        print(bands)
        return imageDF, bands

    return imageDF.mean().values

In [8]:
featuresDF.index

Int64Index([    74,    110,    129,    141,    142,    149,    159,    178,
               201,    213,
            ...
            102113, 102402, 102551, 102565, 102862, 102888, 103030, 103049,
            103051, 103066],
           dtype='int64', length=4170)

In [9]:
imageFeaturesDF = pd.DataFrame(index=featuresDF.index,columns=['features']) 
print(imageFeaturesDF.index)
for row in tqdm(list(featuresDF.index)):
    imageFeatures = featurizeImage(featuresDF.loc[row]['new_filename'])
    imageFeaturesDF.loc[row]['features'] = imageFeatures


Int64Index([    74,    110,    129,    141,    142,    149,    159,    178,
               201,    213,
            ...
            102113, 102402, 102551, 102565, 102862, 102888, 103030, 103049,
            103051, 103066],
           dtype='int64', length=4170)


Exception ignored in: <generator object tqdm_notebook.__iter__ at 0x1134ed468>
Traceback (most recent call last):
  File "/Users/adriencogny/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm_notebook.py", line 187, in __iter__
    yield obj
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
imageFeaturesDF.head()

In [ ]:
imageFeaturesDF.to_csv('image_features_extracted5.csv')

In [10]:
imageFeaturesDF.from_csv('image_features_extracted5.csv')

/Users/adriencogny/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


,features
74,[0.67797597 0.62806031 0.44985327]
110,[0.34375877 0.30431301 0.28888427]
129,[0.69495152 0.68402961 0.70642272]
141,[0.71055381 0.65671828 0.57345266]
142,[0.49535445 0.42498009 0.38846194]
149,[0.55637077 0.61782114 0.66714783]
159,[0.58462249 0.54448039 0.47778 ]
178,[0.66543968 0.72455372 0.75251695]
201,[0.56632862 0.40472872 0.33114008]
213,[0.47377667 0.47981332 0.50116881]


In [13]:
featuresDF['imgFeatures'] = imageFeaturesDF['features']

In [14]:
featuresDF.head()

,new_filename,pixelsx,pixelsy,imgFeatures
74,17354.jpg,5833.0,3985.0,NaN
110,19834.jpg,3701.0,5490.0,NaN
129,15569.jpg,4122.0,4626.0,NaN
141,100478.jpg,3378.0,5448.0,NaN
142,100829.jpg,3672.0,5004.0,NaN


In [ ]:
def compareImages(img1, img2):
    
    diffx = np.abs(img1['pixelsx']-img2['pixelsx']) / (img1['pixelsx'] + img2['pixelsx'])
    diffy = np.abs(img1['pixelsy']-img2['pixelsy']) / (img1['pixelsy'] + img2['pixelsy'])
    
    diffSize = (diffx + diffy)/2
    
    img1Features = img1['imgFeatures']
    img2Features = img2['imgFeatures']
    
    imgFeatureScore = 0
    
    if (len(img1Features) == len(img2Features)):
        imgFeatureDiffs = np.abs(img1Features - img2Features)
        imgFeatureScore = sum(imgFeatureDiffs)
        score = (diffSize+imgFeatureScore)/2
    else:
        score = (diffSize+imgFeatureScore)/2

   
    return 1-score

In [ ]:
def generatePrediction(score,threshold):
    if score > threshold:
        return True
    else:
        return False

def generatePredictions(scores,threshold):
    predictions = []
    for curScore in tqdm(scores,leave=False):
        predictions.append(generatePrediction(curScore,threshold))

    return predictions

In [ ]:
def computeStats(truth,predictions):
    
    accuracy = (predictions == truth).sum()/len(predictions == truth)
    
    trues = [True for _ in range(len(predictions))]
    falses = [False for _ in range(len(predictions))]

    predictionsTrue = [predictions[i] == trues[i] for i in range(len(predictions))]
    predictionsFalse = [predictions[i] == falses[i] for i in range(len(predictions))]
    
    
    truthTrue = [truth[i] == trues[i] for i in range(len(truth))]
    truthFalse = [truth[i] == falses[i] for i in range(len(truth))]
    
#     print(len(predictions))
    truePos = np.sum([predictions[i] == True and truthTrue[i] == True for i in range(len(predictionsTrue))])
#     print(truePos.sum())
    trueNeg = np.sum([predictions[i] == False and truthTrue[i] == False for i in range(len(predictionsFalse))])
    falsePos = np.sum([predictions[i] == True and truthTrue[i] == False for i in range(len(predictionsTrue))])
    falseNeg = np.sum([predictions[i] == False and truthTrue[i] == True for i in range(len(predictionsFalse))])
    
    return accuracy, truePos, trueNeg, falsePos, falseNeg

In [ ]:
def trainThreshold(featuresTrain,maxNumIterations = 100,initialProb = 0.5, increaseRate = 0.001):
    
    probThreshold = initialProb
    prevAccuracy = 0

    for i in tqdm(range(maxNumIterations),leave=False):
        # the score represents how similar two images are

        scores = []
        iloc1 = featuresTrain.sample(frac=0.5).index
        iloc2 = featuresTrain.sample(frac=0.5).index
        # print(len(iloc2))
        # print(nTrials)

        for i, loc in tqdm(enumerate(iloc1),leave=False):
            scores.append(compareImages(featuresTrain.loc[loc],featuresTrain.loc[iloc2[i]]))

        filenames1 = featuresTrain.loc[iloc1]['new_filename'].values
        filenames2 = featuresTrain.loc[iloc2]['new_filename'].values


        truth = trainInfo[trainInfo['filename'].isin(filenames1)]['artist'].values == trainInfo[trainInfo['filename'].isin(filenames2)]['artist'].values
        predictions = generatePredictions(scores,probThreshold)

        #compute true positive -
        accuracy, tp, tn, fp, fn = computeStats(truth,predictions)
       
        if (accuracy < prevAccuracy):
            print("peak accuracy")
            return probThreshold, prevAccuracy,tp, tn, fp, fn
        
        probThreshold += increaseRate
    
    return probThreshold, accuracy, tp, tn, fp, fn

In [ ]:
def computeDevAcc(featuresTrain,probability):

    # the score represents how similar two images are
    scores = []
    iloc1 = featuresTrain.sample(frac=0.5).index
    iloc2 = featuresTrain.sample(frac=0.5).index

    # print(len(iloc2))
    # print(nTrials)

    for i, loc in tqdm(enumerate(iloc1),leave=False):
        scores.append(compareImages(featuresTrain.loc[loc],featuresTrain.loc[iloc2[i]]))

    filenames1 = featuresTrain.loc[iloc1]['new_filename'].values
    filenames2 = featuresTrain.loc[iloc2]['new_filename'].values


    truth = trainInfo[trainInfo['filename'].isin(filenames1)]['artist'].values == trainInfo[trainInfo['filename'].isin(filenames2)]['artist'].values
    predictions = generatePredictions(scores,probability)

    #compute stats
    accuracy, tp, tn, fp, fn = computeStats(truth,predictions)
    return accuracy, tp, tn, fp, fn

In [ ]:
nTrainFraction = 0.6
nDevFraction = 0.4

thresholds = []
devAccuracies = []
tp = []
tn = []
fp = []
fn = []

numberSamples = 10
for i in tqdm(range(numberSamples)):
    featuresTrain = featuresDF.sample(frac=nTrainFraction)
    featuresDev = featuresDF.drop(featuresTrain.index)
#     featuresTest = featuresDF.drop(featuresDev.index).drop(featuresTrain.index).sample(n=nTest)
    learnedThreshold, trainAccuracy, traintp, traintn, trainfp, trainfn = trainThreshold(featuresTrain,50,0.5,0.01)
    thresholds.append((learnedThreshold,trainAccuracy,traintp, traintn, trainfp, trainfn))
    meanDevAccuracy = 0
    accuraciesDev = []
    for i in range(10):
        acc, ctp, ctn, cfp, cfn = computeDevAcc(featuresDev,learnedThreshold)
        accuraciesDev.append(acc)
        tp.append(ctp)
        tn.append(ctn)
        fp.append(cfp)
        fn.append(cfn)
    meanDevAccuracy = np.mean(accuraciesDev)
    devAccuracies.append(meanDevAccuracy)
    
print(thresholds)
print(devAccuracies)
print(tp)
print(tn)
print(fp)
print(fn)

In [ ]:
print(np.mean(tp))
print(np.mean(tn))
print(np.mean(fp))
print(np.mean(fn))

In [ ]:
print(np.mean(devAccuracies))

In [ ]:
np.mean(tn)/(np.mean(fp)+np.mean(tn)) #true negative rate

In [ ]:
(np.mean(tp)+np.mean(tn))/(np.mean(fp)+np.mean(fn) + np.mean(tp)+np.mean(tn)) # accuracy score

In [ ]:
featuresDF.shape